---
title: Asymmetric Key Cipher
---

::::{attention}
This notebook is optional and NOT required for any course assessment activities. Lab tutor may go through them if time is available.
::::

In [ ]:
from __init__ import install_dependencies

await install_dependencies()

In [ ]:
from math import gcd
from random import randint

## Motivation

For symmetric key cipher to work, a secret key needs to be exchanged between a sender and a receiver. Isn't this is a chicken-and-egg problem? *How to securely communicate a secret key in public* to begin with?

This motivates the idea of using [asymmetric keys](https://en.wikipedia.org/wiki/Public-key_cryptography) for encryption and decryption:[^DH] 

[^DH]: See [Diffie and Hellman (1976)](https://doi.org/10.1145/3549993.3550007).

- A sender uses the public key $k$ to encrypt $x$ to a ciphertext $y$ using an encryption function $f$:

    $$
    y = f_k(x).
    $$ (eq:encrypt)

- A receiver uses a private key $l$ to decrypt $y$ back to $x$ using a decryption function $g$:

    $$
    x = g_{l}(y).
    $$ (eq:decrypt)

- To ensure secrecy, anyone with only the knowledge of $k$, $f$, $g$, and $y$ but NOT the knowledge of $l$ should not be able to invert $f_k$.

::::{seealso}

The advantage of using asymmetric key ciphers over symmetric key ciphers lies in their simplicity for:

- **Key sharing**: The encryption key can be publicly shared with all senders.
- **Key storage**: Only the receiver needs to securely store the private key.

Due to the above benefits, a public key cipher system such as RSA is often used to establish a shared secret key, which is then used for symmetric encryption such as [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard). See also [HTTPS](https://en.wikipedia.org/wiki/HTTPS) and [TLS](https://en.wikipedia.org/wiki/Transport_Layer_Security) used on the internet.

::::

## Formulation

**Is it possible to construct an asymmetric key cipher?**

Hopefully, but there is an obvious security issue:

::::{exercise}
:label: ex:rsa2

Prove that $f_k$ is invertible by [](#eq:encrypt) and [](#eq:decrypt). In other words, anyone can recover the plaintext $x$ from its ciphertext $y$ by computing the inverse

$$
x = f_k^{-1}(y),
$$

even without the knowledge of the private key $d$.

::::

YOUR ANSWER HERE

In essence, $f_k$ must be invertible for the ciphertext to be decryptable. To overcome this limitation, we can adopt a different approach. Instead of requiring $f_k$ to be invertible, we can make it *computationally infeasible* to invert. For instance:

::::{prf:example} Integer factorization

Computing the product $n$ of from two prime numbers $p$ and $q$, i.e., $n=pq$, is easy. However, the reverse process of factoring $n$ to $p$ and $q$ is difficult (in a class of problems called [co-NP](https://en.wikipedia.org/wiki/Integer_factorization)). As the size of $p$ and $q$ increases, the time required to factor $n$ increases dramatically as illustrated [here](https://www.khanacademy.org/computing/computer-science/cryptography/modern-crypt/pi/time-complexity-exploration).

::::

We seek a collection of $f_k$'s that are easy to compute in one way but difficult to invert directly except using $g_l$. Such a function is called a trapdoor function, which is a special kind of [one-way function](https://en.wikipedia.org/wiki/One-way_function):

::::{prf:definition} Public/asymmetric key cipher
:label: def:akc

A public key cipher is a family of [trapdoor functions](https://en.wikipedia.org/wiki/Trapdoor_function#References) 

$$
f_k: \mathcal{X} \to \mathcal{Y}
$$ 

pameterized by the public key $k\in \mathcal{K}$. In particular, a sequence of such cipher with respect to the key length $m:=\lceil \log \lvert \mathcal{K}\rvert \rceil$ is:[^m]
- **Encryptable** if computing $y := f_k(x)$ is probabilistic polynomial time ([PP](https://en.wikipedia.org/wiki/PP_(complexity))) in $m$ for every $x\in \mathcal{X}$.
- **Decryptable** if there exists $g_l$ for each $f_k$ such that $g_l(y) = x$ almost surely and can be done in PP.
- **Secure** if computing $x$ from $y$, $l$, $f$, and $g$ is not PP.

::::

[^m]: Computational complexity is an asymptotic notion with respect to the order of growth of $m$.

**How to construct the trapdoor function $f_k$ and the inverse $g_l$?**

No one knows:

::::{seealso} P vs NP

The existence of one-way functions remains unknown today. If one could show existence of such functions, they would prove [P$\neq $NP](https://en.wikipedia.org/wiki/P_versus_NP_problem), resolving one of the [Millennium Prize Problems](https://en.wikipedia.org/wiki/Millennium_Prize_Problems).

::::

Nevertheless, there are plausible implementations such as the RSA algorithm.

## RSA Algorithm

In the sequel, we will dive into the RSA algorithm, which was a public key cipher invented by [Rivest, Shamir, and Adleman](https://doi.org/10.1145/359340.359342) at MIT. Its security is based on the computational difficulty of factoring numbers with prime factors.

### Encryption and Decryption

The encryption and decryption use modular exponentiation instead of addition:

::::{prf:definition} RSA encryption/decryption
:label: def:rsa

$$
\begin{align}
f_k(x) &:= x^e \bmod n && \text{where }k:=(e,n)\\
g_l(y) &:= y^d \bmod n && \text{where }l:=(d,n),
\end{align}
$$

where $e$, $d$, and $n$ are positive integers, and 

$$
x, y \in \mathcal{X} = \mathcal{Y} := \Set{1, \dots, n-1}.
$$

::::

$e$, $d$, and $n$ need to be chosen carefully to ensure security. For instance, it is insecure to choose $e=1$ because, otherwise, $x=f_k(x)$, i.e., the plaintext is unencrypted, or equivalently, the private key is readily computed with $d=1$.

::::{exercise}
:label: ex:zero-excluded

Why is it insecure to allow the plaintext $x$ to be $0$, i.e., with $\mathcal{X}:=\Set{0, 1, \dots, n-1}$ instead?

::::

YOUR ANSWER HERE

Computing the exponentiation for encryption and decryption can be fast using [repeated squaring](https://en.wikipedia.org/wiki/Exponentiation_by_squaring). The built-in function `pow` already has an efficient implementation:

In [ ]:
x, e, n = 3, 2 ** 1000000, 4
pow(x, e, n)

Is `pow(x, e, n)` the same as the following code?

```python
x ** e % n
```

Try it in a new console.

::::{exercise}
:label: ex:rsa3

Implement your own code `modular_pow` (without using the `pow` function) using [repeated squaring](https://en.wikipedia.org/wiki/Exponentiation_by_squaring) and the property that

$$
(a^c \bmod b) = (a\bmod b)^c \bmod b.
$$ (eq:rsa:modular_pow)

::::

In [ ]:
def mod_pow(x, e, n):
    """
    Return mod exponentiation (x ** e) % n efficiently.
    """
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
x, e, n = 3, 2 ** 100000, 4
assert mod_pow(x, e, n) == pow(x, e, n)

Is that all to RSA algorithm? Not really... The question is how to choose the integers $e$, $d$, and $n$ properly so that the cipher is both decryptable and secure.

### Decryptability

To ensure decryptability, we require for all $x\in \mathcal{X}$ that

$$
\begin{align}
x^{ed} &\equiv x  \mod n.
\end{align} 
$$ (eq:rsa:decryptability)

::::{exercise}
:label: ex:rsa4

Derive the above condition [](#eq:rsa:decryptability) using the property [](#eq:rsa:modular_pow).

::::

::::{solution} ex:rsa4
:class: dropdown

:::{prf:proof}
:nonumber:

$$
\begin{align}
x &= g_l(f_k(x)) \\
&= (x^{e} \bmod n)^{d} \bmod n \\
&= x^{ed} \bmod n. \qquad (Q.E.D.)
\end{align}
$$

:::

::::

RSA makes use of the following result to choose $(e, d, n)$:

::::{prf:proposition} Fermat's little theorem
:label: pro:fermat

For any prime $p$ and integer $x$,

$$
x^{p}\equiv x \mod p
$$ (eq:fermat)

::::

There are elegant and elementary [combinatorial proofs](https://en.wikipedia.org/wiki/Proofs_of_Fermat%27s_little_theorem#Combinatorial_proofs) of the theorem. A simply algebraic proof is as follows.

::::{prf:proof} [](#pro:fermat)
:nonumber:
:class: dropdown

Consider the non-trivial case where $x$ is not divisible by $p$. It follows that, for $i, j \in \Set{1, \dots, p-1}$,

$$
x\cdot i \equiv x\cdot j \mod p \iff i \equiv j \mod p,
$$

which, in turn, implies

\begin{align}
\prod_{i=1}^{p-1} (x\cdot i) &\equiv  \overbrace{\prod_{i=1}^{p-1} i}^{\alpha:=} \mod p\\
x^{p-1} \alpha  &\equiv \alpha \mod p,
\end{align}

which implies [](#eq:fermat) as desired by canceling the non-zero value $\alpha$ on both sides. (Q.E.D.)

::::

Comparing [](#eq:fermat) in Fermat's little theorem to the descryptability condition [](eq:rsa:decryptability), it appears we can choose

$$
n = p = ed.
$$

Not really... because, otherwise, either $e$ or $d$ is $1$, i.e., the private key can be easily computed from the public key. (How?)

Consider the following corollary, which follows from [](#eq:fermat):

::::{prf:corollary}
:label: cor:fermat-m

For any prime $p$ and integers $m$ and $x\in \Set{1,\dots, p-1}$,

$$
x^{m(p-1)} \equiv 1 \mod p.
$$ (eq:fermat-m)


::::

::::{exercise}
:label: ex:fermat-m

Prove the above corollary.

::::

::::{solution} ex:fermat-m
:class: dropdown

:::{prf:proof} [](#cor:fermat-m)
:nonumber:

Since $p$ does not divide $x$, dividing $x$ from both sides of [](#eq:fermat) gives

\begin{align}
x^{p-1} &\equiv 1 \mod p.
\end{align}

Raising both sides by a power of $m$ gives [](#eq:fermat-m) as desired.


:::

::::

We can similarly rewrite [](eq:rsa:decryptability) as:

$$
x^{ed-1} \equiv 1 \mod p
$$ (eq:rsa:decryptability2)

for $x\in \Set{1,\dots, p-1}$.

Comparing with [](#eq:fermat-m) with [](#eq:rsa:decryptability2), can we have choose

$$
n = p \kern1em \text{ and } \kern1em ed \equiv 1 \bmod p-1?
$$

No because $d$ is the [modular multiplicative inverse](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse) of $e$, which is easy to compute, e.g., using `pow` with an exponent of `-1`. For instance:

In [ ]:
p = 7
e, n = 5, p
try:
    d = pow(e, -1, n - 1)    # easy to compute
    x = randint(1, n-1)      # plaintext
    y = pow(x, e, n)         # encryption
    assert x == pow(y, d, n) # decryption
except ValueError as e:
    print(e)

Note also that some choice of $e$ does not have a multiplicative inverse. E.g, try setting $e$ to be $0$, $2$, $3$, $4$.

::::{prf:proposition} Multiplicative inverse
:label: pro:mod_inverse

$e$ is coprime to $\lambda$, i.e., 

$$
\operatorname{gcd}(e, \lambda)=1,
$$ (eq:coprime)

iff it has a modular multicative inverse with respect to $\lambda$, i.e., there exists $d$ such that

$$
ed \equiv 1 \mod \lambda.
$$ (eq:mod_inverse)

::::

::::{prf:proof} [](#pro:mod_inverse)
:nonumber:
:class: dropdown

By the [Euclidean algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm),

$$
ed = m\lambda + \operatorname{gcd}(e, \lambda) \kern1em \text{ for some integers $m$ and $d$,}
$$

which is called [Bézout's identity](https://en.wikipedia.org/wiki/B%C3%A9zout%27s_identity). Equivalently,

$$
ed \equiv \operatorname{gcd}(e, \lambda) \mod \lambda \kern1em \text{ for some integers $d$,}
$$

which implies [](#eq:coprime) is equivalent to [](#eq:mod_inverse) as desired. (Q.E.D.)

::::

::::{exercise}
:label: ex:mod_inverse

Implement your own modular inverse using the [Extended Euclidean Algorithm](https://en.wikipedia.org/wiki/Modular_multiplicative_inverse#Computation) instead of `pow`.

::::

In [ ]:
def mod_inverse(e, n):
    """
    Return the modular multiplicative inverse of e mod n, or
    None if no inverse exists.
    """
    # YOUR CODE HERE
    raise NotImplementedError

In [ ]:
# tests
p = 7
assert [mod_inverse(e, p - 1) for e in range(p - 1)] == [None, 1, None, None, None, 5]

**How to make it difficult to compute $d$?**

RSA makes use of the hardness of factoring a product of large primes to create the desired trapdoor function.

::::{prf:theorem} RSA
:label: thm:rsa

For the product $n:=pq$ of any prime numbers $p$ and $q$,

$$
x^{m \lambda(n)} \equiv 1 \mod n
$$ (eq:rsa)

for all $x\in \Set{1,\dots, n-1}$, where 

$$
\lambda(n) := \operatorname{lcm}(p-1, q-1),
$$ (eq:lambda)

called the [Carmichael function](https://en.wikipedia.org/wiki/Carmichael_function), is the least common multiple (lcm) of $p-1$ and $q-1$. 

::::

::::{prf:proof} [](#thm:rsa)
:nonumber:
:class: dropdown

With $m(p-1)$ in [](#eq:fermat-m) being the least common multiple $\operatorname{lcm}(p-1,q-1)$ for another prime number $q$, we have

$$
\begin{align}
x^{\operatorname{lcm}(p-1, q-1)} &\equiv 1 \mod p && \text{and}\\
x^{\operatorname{lcm}(p-1, q-1)} &\equiv 1 \mod q && \text{by symmetry.}
\end{align}
$$

This implies $x^{\operatorname{lcm}(p-1, q-1)} - 1$ is divisible by both $p$ and $q$, and so

$$
x^{\operatorname{lcm}(p-1, q-1)} \equiv 1 \mod p q.
$$

Raising both sides to the power of any positive integer $m$ gives [](#eq:rsa).

::::

::::{prf:definition} RSA key generation
:label: def:rsa-key

To generate the public and private keys for the RSA encryption and decryption in [](#def:rsa):

1. Generate two distinct large prime numbers $p$ and $q$ and set $n := pq$.
2. Choose $e$ from $\mathcal{K}$ defined to be the subset of numbers from $\Set{3,\dots,\lambda(n)-1}$ coprime to $\lambda(n)$ defined in [](#eq:lambda).
3. Compute $d$ from $\mathcal{K}$ as the modular multiplicative inverse of $e$ with respect to $\lambda(n)$ as in [](#eq:mod_inverse).

::::

The choice of $e$ guarantees a multiplicative inverse $d$ exists by [](#pro:mod_inverse). Note that $2$ is excluded from $\mathcal{K}$ because $\lambda(n)$ must be an even number. (Why?)

::::{exercise}
:label: ex:rsa:decryptability

Prove that RSA is decryptable [](#eq:rsa:decryptability) with the choice of the keys in [](#def:rsa-key).

::::

::::{solution} ex:rsa:decryptability
:class: dropdown

:::{prf:proof}
:nonumber:

By [](#eq:mod_inverse),

\begin{align}
ed &\equiv 1 \mod \lambda(n)\\
ed - 1 &= m\lambda(n) &&\text{for some integer $m$}\\
\end{align}
$$ (ed)

which implies [](#eq:rsa:decryptability2) by [](#eq:rsa).

Furthermore, computing $d$ is as hard as computing $\operatorname{lcm}(p-1, q-1)$ or factoring $n$.
:::
::::

::::{seealso} Is it secure?

Although $d$ can be computed as the modular multiplicative inverse of $e$, it is with respect to $\operatorname{lcm}(p-1, q-1)$, which is not easy to compute without knowing the factors of $n$, namely $p$ and $q$. It was shown in the [original paper](https://doi.org/10.1145/359340.359342) that computing $d$ is as hard as computing $\operatorname{lcm}(p-1, q-1)$ or factoring $n$.

::::

**How to generate the public key and private key?**

By {eq}`ed`, we can compute $d$ as the modular multiplicative inverse of $e$. How to choose $e$ then?

We can choose any $e \in \{1, \dots, \operatorname{lcm}(p-1, q-1)\}$ such that $e$ does not divide $\operatorname{lcm}(p-1, q-1)$.

::::{exercise}
:label: ex:get_rsa_keys

Complete the following function to randomly generates the `e, d, n` for some given prime numbers `p` and `q`.

:::{hint}
:class: dropdown

Note that

$$
\operatorname{lcm}(p-1, q-1) = \frac{(p-1)(q-1)}{\operatorname{gcd}(p-1, q-1)}.
$$

:::

::::

In [ ]:
from math import gcd
from random import randint


def get_rsa_keys(p, q):
    n = p * q
    lcm = (p - 1) * (q - 1) // gcd(p - 1, q - 1)
    while True:
        e = randint(2, lcm - 1)
        # YOUR CODE HERE
        raise NotImplementedError
    d = pow(e, -1, lcm)
    return e, d, n, lcm

As an example, if we choose two prime numbers $p=17094589121$ and $q=1062873761$:

In [ ]:
e, d, n, lcm = get_rsa_keys(17094589121, 1062873761)
assert e * d % lcm == 1
print(
    f"""Public key : k = (e, n) = ({e}, {n})
Private key: l = (d, n) = ({d}, {n})"""
)

The integer $1302$ can be encrypted using the public key as follows:

In [ ]:
x = 1302
y = pow(x, e, n)
print(f'The plaintext {x} gets encrypted into {y}.')

With the private key, the ciphertext can be decrypted easily as follows:

In [ ]:
output = pow(y, d, n)
print(f'The ciphertext {y} gets decrypted into {output}.')

To complete the implementation of RSA, we need to generate large prime numbers. Computing large prime numbers or testing the primality of a large number is difficult. (See the [largest known prime number](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent).) Fortunately, there are approximate primality test that is fast, e.g., the [Rabin-Miller test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test). An implementation is available from `sympy`:

In [ ]:
import sympy as sp

x = 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127
sp.isprime(x)

As the docstring of [`isprime`](https://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.primetest.isprime) mentioned, there is a small chance of false positive, but this is acceptable as there is no known cases.

```
Test if n is a prime number (True) or not (False). For n < 2^64 the
answer is definitive; larger n values have a small probability of actually
being pseudoprimes.

Negative numbers (e.g. -2) are not considered prime.

The first step is looking for trivial factors, which if found enables
a quick return.  Next, if the sieve is large enough, use bisection search
on the sieve.  For small numbers, a set of deterministic Miller-Rabin
tests are performed with bases that are known to have no counterexamples
in their range.  Finally if the number is larger than 2^64, a strong
BPSW test is performed.  While this is a probable prime test and we
believe counterexamples exist, there are no known counterexamples.
```

::::{seealso} RSA package

There is a module called `rsa` to [generate a RSA key pair](https://stuvel.eu/python-rsa-doc/usage.html#generating-keys). You can also use `ssh-keygen` command in a linux terminal to generate an RSA key for [secure remote access to a server such as GitHub](https://docs.github.com/en/authentication/connecting-to-github-with-ssh/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent).

::::